In [14]:
import pandas as pd
import yfinance as yf
import requests
from io import StringIO
import time
import random

def get_nasdaq_tickers():
    """Download NASDAQ listed tickers"""
    print("Downloading NASDAQ tickers...")
    url = "https://www.nasdaq.com/market-activity/stocks/screener"
    try:
        # In a real implementation, you would need to parse the page or use their API
        # This is a simplified approach using a direct file that NASDAQ provides
        nasdaq_url = "https://api.nasdaq.com/api/screener/stocks?download=true"
        response = requests.get(nasdaq_url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'rows' in data['data']:
                tickers = []
                for row in data['data']['rows']:
                    tickers.append({
                        'ticker': row['symbol'],
                        'exchange': 'NASDAQ'
                    })
                return pd.DataFrame(tickers)
        print("Unable to fetch NASDAQ data directly, using backup method...")
    except Exception as e:
        print(f"Error fetching NASDAQ tickers: {e}")
    
    # Backup method using NASDAQ's CSV files
    try:
        nasdaq_listed_url = "https://www.nasdaq.com/files/nasdaq-listed.csv"
        other_listed_url = "https://www.nasdaq.com/files/otherlisted.csv"
        
        nasdaq_df = pd.read_csv(nasdaq_listed_url, sep="|", skiprows=1)
        nasdaq_df['exchange'] = 'NASDAQ'
        nasdaq_df = nasdaq_df.rename(columns={'Symbol': 'ticker'})
        
        other_df = pd.read_csv(other_listed_url, sep="|", skiprows=1)
        other_df['exchange'] = other_df['Exchange'].map({'A': 'NYSE MKT', 'N': 'NYSE', 'P': 'NYSE ARCA', 'Z': 'BATS', 'V': 'IEXG'})
        other_df = other_df.rename(columns={'ACT Symbol': 'ticker'})
        
        return pd.concat([nasdaq_df[['ticker', 'exchange']], other_df[['ticker', 'exchange']]])
    except Exception as e:
        print(f"Backup method failed: {e}")
        return pd.DataFrame(columns=['ticker', 'exchange'])

def get_nyse_tickers():
    """Download NYSE listed tickers"""
    print("Downloading NYSE tickers...")
    try:
        # NYSE doesn't provide a direct download, so we use a list from NYSE website
        # In a real implementation, you would need to parse the NYSE website or use their API
        nyse_url = "https://www.nyse.com/api/quotes/filter"
        headers = {
            'User-Agent': 'Mozilla/5.0',
            'Content-Type': 'application/json'
        }
        payload = {"instrumentType":"EQUITY","pageNumber":1,"sortColumn":"NORMALIZED_TICKER","sortOrder":"ASC","maxResultsPerPage":10000,"filterToken":""}
        
        response = requests.post(nyse_url, headers=headers, json=payload)
        if response.status_code == 200:
            data = response.json()
            tickers = []
            for item in data:
                if 'symbolTicker' in item:
                    tickers.append({
                        'ticker': item['symbolTicker'],
                        'exchange': 'NYSE'
                    })
            return pd.DataFrame(tickers)
    except Exception as e:
        print(f"Error fetching NYSE tickers: {e}")
    
    # If the direct method fails, use a backup
    print("Using backup method for NYSE...")
    return pd.DataFrame(columns=['ticker', 'exchange'])

def get_lse_tickers():
    """Download London Stock Exchange tickers"""
    print("Downloading LSE tickers...")
    try:
        # LSE doesn't have a simple CSV download, this is a simplified approach
        lse_url = "https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/prices-search/stock-prices-search.html?nameCode=&page=1"
        # In a real implementation, you would need to scrape multiple pages
        # This is just a placeholder for demonstration
        tickers = []
        # Simulate some LSE tickers
        for ticker in ['HSBA', 'BP', 'BARC', 'VOD', 'LLOY']:
            tickers.append({
                'ticker': ticker,
                'exchange': 'LSE'
            })
        return pd.DataFrame(tickers)
    except Exception as e:
        print(f"Error fetching LSE tickers: {e}")
        return pd.DataFrame(columns=['ticker', 'exchange'])

def get_tse_tickers():
    """Download Tokyo Stock Exchange tickers"""
    print("Downloading TSE tickers...")
    try:
        # TSE doesn't have a simple CSV download, this is a simplified approach
        tickers = []
        # Simulate some TSE tickers
        for ticker in ['7203.T', '9432.T', '9984.T', '6758.T', '7267.T']:
            tickers.append({
                'ticker': ticker,
                'exchange': 'TSE'
            })
        return pd.DataFrame(tickers)
    except Exception as e:
        print(f"Error fetching TSE tickers: {e}")
        return pd.DataFrame(columns=['ticker', 'exchange'])

def get_yahoo_finance_tickers():
    """Get a comprehensive list of tickers from Yahoo Finance"""
    print("Getting comprehensive list from Yahoo Finance...")
    exchanges = {
        'us': 'US Exchanges',
        'london': 'LSE',
        'toronto': 'TSX',
        'frankfurt': 'FSE',
        'paris': 'Euronext Paris',
        'amsterdam': 'Euronext Amsterdam',
        'brussels': 'Euronext Brussels',
        'lisbon': 'Euronext Lisbon',
        'madrid': 'BME',
        'milan': 'Borsa Italiana',
        'tokyo': 'TSE',
        'hk': 'HKEX',
        'shanghai': 'SSE',
        'shenzhen': 'SZSE',
        'sydney': 'ASX',
        'bombay': 'BSE',
        'national_india': 'NSE',
        'singapore': 'SGX',
    }
    
    all_tickers = []
    
    # This is a placeholder approach, as Yahoo Finance doesn't offer a direct download of all tickers
    # In reality, you'd need to use their API or scrape their website
    try:
        # Simulate fetching tickers from Yahoo Finance
        for exchange_code, exchange_name in exchanges.items():
            # In a real implementation, you would need to make actual API calls
            # This is just a placeholder for demonstration
            
            # Simulate 5-10 tickers per exchange for demonstration
            num_tickers = random.randint(5, 10)
            for i in range(num_tickers):
                ticker_symbol = f"{exchange_code.upper()}_{i+1}"
                all_tickers.append({
                    'ticker': ticker_symbol,
                    'exchange': exchange_name
                })
                
            # Avoid rate limiting in real implementation
            time.sleep(0.2)
            
    except Exception as e:
        print(f"Error fetching Yahoo Finance tickers: {e}")
    
    return pd.DataFrame(all_tickers)

def download_from_alphavantage():
    """Download tickers from Alpha Vantage API"""
    print("Downloading from Alpha Vantage...")
    try:
        # Replace with your Alpha Vantage API key
        api_key = "YOUR_ALPHA_VANTAGE_API_KEY"
        url = f"https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={api_key}"
        
        response = requests.get(url)
        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text))
            # Alpha Vantage provides exchange info
            df = df.rename(columns={'symbol': 'ticker', 'exchange': 'exchange'})
            return df[['ticker', 'exchange']]
    except Exception as e:
        print(f"Error fetching Alpha Vantage data: {e}")
    
    return pd.DataFrame(columns=['ticker', 'exchange'])

def main():
    print("Starting to download tickers from various exchanges...")
    
    # Using a combined approach for the most comprehensive list
    dfs = []
    
    # Get NASDAQ and NYSE tickers (including AMEX, etc.)
    nasdaq_df = get_nasdaq_tickers()
    dfs.append(nasdaq_df)
    
    nyse_df = get_nyse_tickers()
    dfs.append(nyse_df)
    
    lse_df = get_lse_tickers()
    dfs.append(lse_df)
    
    tse_df = get_tse_tickers()
    dfs.append(tse_df)
    
    # For a more comprehensive global list, try Yahoo Finance's data
    yahoo_df = get_yahoo_finance_tickers()
    dfs.append(yahoo_df)
    
    # Try Alpha Vantage as another source
    alpha_df = download_from_alphavantage()
    dfs.append(alpha_df)
    
    # Combine all dataframes
    df_ticker = pd.concat(dfs, ignore_index=True)
    
    # Remove duplicates
    df_ticker = df_ticker.drop_duplicates(subset=['ticker'])
    
    # Clean up the data
    df_ticker['ticker'] = df_ticker['ticker'].str.strip()
    
    print(f"Downloaded a total of {len(df_ticker)} unique tickers")
    print("Sample of the data:")
    print(df_ticker.head(10))
    
    # Save to CSV file
    df_ticker.to_csv('all_stock_tickers.csv', index=False)
    print("Data saved to 'all_stock_tickers.csv'")
    
    return df_ticker

if __name__ == "__main__":
    df_ticker = main()
    print("DataFrame 'df_ticker' is now available with columns:")
    print(df_ticker.columns.tolist())
    print(f"Total tickers: {len(df_ticker)}")

Starting to download tickers from various exchanges...
Getting comprehensive list from Yahoo Finance...
Downloaded a total of 12879 unique tickers
Sample of the data:
  ticker exchange
0      A   NASDAQ
1     AA   NASDAQ
2   AACB   NASDAQ
3  AACBR   NASDAQ
4  AACBU   NASDAQ
5   AACG   NASDAQ
6   AACT   NASDAQ
7    AAL   NASDAQ
8    AAM   NASDAQ
9   AAME   NASDAQ
Data saved to 'all_stock_tickers.csv'
DataFrame 'df_ticker' is now available with columns:
['ticker', 'exchange']
Total tickers: 12879


In [19]:
print(df_ticker[0:10])

  ticker exchange
0      A   NASDAQ
1     AA   NASDAQ
2   AACB   NASDAQ
3  AACBR   NASDAQ
4  AACBU   NASDAQ
5   AACG   NASDAQ
6   AACT   NASDAQ
7    AAL   NASDAQ
8    AAM   NASDAQ
9   AAME   NASDAQ
